## ETL -2

In [1]:
import requests
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

In [4]:
df = pd.read_csv('../datos/df_ccaa_year.csv', index_col=0)

In [5]:
df.sample(4)

,value,percentage,datetime,type,CCAA
1,2.760811e+07,0.715143,2011-01-01T00:00:00.000+01:00,no renovable,Andalucía
324,1.797966e+07,0.742584,2021-01-01T00:00:00.000+01:00,renovable,Galicia
148,1.218521e+06,0.651413,2015-01-01T00:00:00.000+01:00,renovable,La Rioja
123,7.687566e+06,0.493016,2015-01-01T00:00:00.000+01:00,no renovable,Aragón


In [6]:
df_censal = pd.read_csv('../datos/poblacion_comunidades.csv', index_col=0)

In [7]:
df_censal.sample(4)

,Comunidad,indice,Comunidades_y_Ciudades_Autónomas,Año,Total
18,Ceuta,8744,Ceuta,2004,74.654
238,Principado de Asturias,11,Principado de Asturias,2009,1.085.289
15,Ceuta,8744,Ceuta,2007,76.603
317,Comunitat Valenciana,15,Comunitat Valenciana,2005,4.692.449


En la lección de hoy aprendimos como transformar nuestros datos para que estén preparados para almacearlos en una BBDD. En este momento tenemos dos fuentes de datos:

El csv donde tenemos la información censal de cada Comunidad Autónoma tiene algunas columnas que no son redundantes para nuestro análisis, en este csv tendremos que:

Eliminar la columna de Comunidades_y_Ciudades_Autónomas

En el csv con la información de generación de energías renovables y no renovables a nivel nacional deberéis:
Para las columnas value y percentage, redondear los decimales de los valores a 2.
La columna datetime es de tipo string, pero esto a nosotras no nos interesa. Además, en esta columna tiene tanto la fecha como la hora, pero en realidad la hora no nos esta aportando nada de información, es por esto que:
Deberéis crear una columna nueva donde solo tengamos la información de la fecha en formato AÑO / MES / DIA.
Una vez creada la columna anterior, convertirla a datetime.

En el csv con la información de generación de energías renovables y no renovables a nivel de comunidad autónoma deberéis aplicar los mismos cambios que aplicamos en el ejercicio anterior.


In [8]:
#definimos función para borrar columnas
def borrar_columnas(df,col):
    df.drop(columns = col, axis = 1, inplace = True)
    return df

In [9]:
borrar_columnas(df_censal, "Comunidades_y_Ciudades_Autónomas")

,Comunidad,indice,Año,Total
0,Ceuta,8744,2022,83.117
1,Ceuta,8744,2021,83.517
2,Ceuta,8744,2020,84.202
3,Ceuta,8744,2019,84.777
4,Ceuta,8744,2018,85.144
...,...,...,...,...
470,La Rioja,20,2002,281.614
471,La Rioja,20,2001,270.400
472,La Rioja,20,2000,264.178
473,La Rioja,20,1999,265.178


In [10]:
df_censal

,Comunidad,indice,Año,Total
0,Ceuta,8744,2022,83.117
1,Ceuta,8744,2021,83.517
2,Ceuta,8744,2020,84.202
3,Ceuta,8744,2019,84.777
4,Ceuta,8744,2018,85.144
...,...,...,...,...
470,La Rioja,20,2002,281.614
471,La Rioja,20,2001,270.400
472,La Rioja,20,2000,264.178
473,La Rioja,20,1999,265.178


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 360 entries, 0 to 359
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   value       360 non-null    float64
 1   percentage  360 non-null    float64
 2   datetime    360 non-null    object 
 3   type        360 non-null    object 
 4   CCAA        360 non-null    object 
dtypes: float64(2), object(3)
memory usage: 16.9+ KB


In [66]:
def limpiar_energia(df):
#esta parte de la función redondea las columnas numéricas
    for col in df.columns:
        if df[col].dtype == float:
            df[col] = round(df[col],2)
        else:
            pass
    return df

In [71]:
def cambiar_fecha(df):
    df["fecha"] = df["datetime"].str.split("T")
    df_fecha = df["fecha"].apply(pd.Series)
    df = df.merge(df_fecha, left_index=True, right_index = True )
    df.drop(columns = [1, "datetime", "fecha"], inplace = True, axis = 1)
    df.rename(columns = {0:"fecha"}, inplace = True)
    df["fecha"] = df["fecha"].apply(pd.to_datetime)
    return df
    
    

In [25]:
df["fecha"] = df["datetime"].str.split("T")


In [41]:
df_fecha = df["fecha"].apply(pd.Series)

In [42]:
borrar_columnas(df_fecha, 1)

,0
0,2011-01-01
1,2011-01-01
2,2011-01-01
3,2011-01-01
4,2011-01-01
...,...
355,2022-01-01
356,2022-01-01
357,2022-01-01
358,2022-01-01


In [45]:
df = df.merge(df_fecha, left_index=True, right_index = True )

In [46]:
df

,value,percentage,datetime,type,CCAA,fecha,0
0,1.099692e+07,0.284857,2011-01-01T00:00:00.000+01:00,renovable,Andalucía,"[2011-01-01, 00:00:00.000+01:00]",2011-01-01
1,2.760811e+07,0.715143,2011-01-01T00:00:00.000+01:00,no renovable,Andalucía,"[2011-01-01, 00:00:00.000+01:00]",2011-01-01
2,7.005216e+06,0.386497,2011-01-01T00:00:00.000+01:00,renovable,Aragón,"[2011-01-01, 00:00:00.000+01:00]",2011-01-01
3,1.111968e+07,0.613503,2011-01-01T00:00:00.000+01:00,no renovable,Aragón,"[2011-01-01, 00:00:00.000+01:00]",2011-01-01
4,3.385157e+05,0.170045,2011-01-01T00:00:00.000+01:00,renovable,Cantabria,"[2011-01-01, 00:00:00.000+01:00]",2011-01-01
...,...,...,...,...,...,...,...
355,7.891030e+06,0.352032,2022-01-01T00:00:00.000+01:00,no renovable,Galicia,"[2022-01-01, 00:00:00.000+01:00]",2022-01-01
356,2.670446e+06,0.229165,2022-01-01T00:00:00.000+01:00,renovable,Región de Murcia,"[2022-01-01, 00:00:00.000+01:00]",2022-01-01
357,8.982495e+06,0.770835,2022-01-01T00:00:00.000+01:00,no renovable,Región de Murcia,"[2022-01-01, 00:00:00.000+01:00]",2022-01-01
358,1.106679e+06,0.476786,2022-01-01T00:00:00.000+01:00,renovable,La Rioja,"[2022-01-01, 00:00:00.000+01:00]",2022-01-01


In [47]:
borrar_columnas(df, ["datetime", "fecha"])


,value,percentage,type,CCAA,0
0,1.099692e+07,0.284857,renovable,Andalucía,2011-01-01
1,2.760811e+07,0.715143,no renovable,Andalucía,2011-01-01
2,7.005216e+06,0.386497,renovable,Aragón,2011-01-01
3,1.111968e+07,0.613503,no renovable,Aragón,2011-01-01
4,3.385157e+05,0.170045,renovable,Cantabria,2011-01-01
...,...,...,...,...,...
355,7.891030e+06,0.352032,no renovable,Galicia,2022-01-01
356,2.670446e+06,0.229165,renovable,Región de Murcia,2022-01-01
357,8.982495e+06,0.770835,no renovable,Región de Murcia,2022-01-01
358,1.106679e+06,0.476786,renovable,La Rioja,2022-01-01


In [50]:
df.rename(columns = {0:"fecha"}, inplace = True)

In [54]:
df["fecha"] = df["fecha"].apply(pd.to_datetime)

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 360 entries, 0 to 359
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   value       360 non-null    float64       
 1   percentage  360 non-null    float64       
 2   type        360 non-null    object        
 3   CCAA        360 non-null    object        
 4   fecha       360 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 25.0+ KB


In [56]:
df.sample(3)

,value,percentage,type,CCAA,fecha
16,5.440115e+05,0.311170,renovable,Comunidad de Madrid,2011-01-01
266,1.880736e+06,0.179588,renovable,Región de Murcia,2019-01-01
70,9.488286e+06,0.210713,renovable,Cataluña,2013-01-01


In [58]:
limpiar_energia(df)

In [59]:
df.sample(3)

,value,percentage,type,CCAA,fecha
89,530714.23,0.27,no renovable,La Rioja,2013-01-01
32,7236334.23,0.43,renovable,Aragón,2012-01-01
202,4764185.59,0.23,renovable,Extremadura,2017-01-01


In [63]:
df_nacional = pd.read_csv('../datos/df_energia_nacional.csv', index_col = 0)
df_nacional.sample(3)

,value,percentage,datetime,type
17,1.607465e+08,0.613921,2016-01-01T00:00:00.000+01:00,no renovable
5,1.010892e+08,0.386079,2016-01-01T00:00:00.000+01:00,renovable
16,1.733657e+08,0.648208,2015-01-01T00:00:00.000+01:00,no renovable


In [64]:
limpiar_energia(df_nacional)

In [65]:
df_nacional.sample(3)

,value,percentage,datetime,type
16,1.733657e+08,0.65,2015-01-01T00:00:00.000+01:00,no renovable
1,8.533872e+07,0.30,2012-01-01T00:00:00.000+01:00,renovable
12,1.926868e+08,0.69,2011-01-01T00:00:00.000+01:00,no renovable


In [72]:
cambiar_fecha(df_nacional)

,value,percentage,type,fecha
0,8.666705e+07,0.31,renovable,2011-01-01
1,8.533872e+07,0.30,renovable,2012-01-01
2,1.097577e+08,0.40,renovable,2013-01-01
3,1.078761e+08,0.40,renovable,2014-01-01
4,9.408811e+07,0.35,renovable,2015-01-01
5,1.010892e+08,0.39,renovable,2016-01-01
6,8.461087e+07,0.32,renovable,2017-01-01
7,1.003444e+08,0.38,renovable,2018-01-01
8,9.791330e+07,0.38,renovable,2019-01-01
9,1.106052e+08,0.44,renovable,2020-01-01


In [73]:
df_nacional.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 0 to 23
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   value       24 non-null     float64
 1   percentage  24 non-null     float64
 2   datetime    24 non-null     object 
 3   type        24 non-null     object 
 4   fecha       24 non-null     object 
dtypes: float64(2), object(3)
memory usage: 1.7+ KB



Guardar los resultados obtenidos en pickle que usaremos en próximos ejercicios de pair programming. Cada fichero debe ir en un pickle

In [61]:
df_censal.to_csv('../datos/poblacion_comunidades_clean.csv')

In [62]:
df.to_csv('../datos/df_ccaa_clean.csv')